# 1.1 Neural Network SMS Text Classifier

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement, or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or " spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  %pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
%pip install tensorflow-datasets
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.4/588.4 MB 3.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.4/439.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 70.3 MB/s eta 0:00:00:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


2022-10-16 16:05:14.207920: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-16 16:05:17.757799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-16 16:05:17.757834: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-16 16:05:18.047791: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-16 16:05:24.096045: W tensorflow/stream_executor/pla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 57.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 kB 11.0 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=a129fcfaa894c25eedb67d9ea4806f38f6d4c46613623386211bf02d9a665f9a
  Stored in directory: /home/codespace/.cache/pip/wheels/54/4e/28/3ed0e1c8a752867445bab994d2340724928aa3ab059c57c8db
Successfully built promise

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.10.0


### 1.2 Loading the SMS Spam Collection dataset

In [2]:
# Get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-10-16 16:05:52--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K   206KB/s    in 1.7s    

2022-10-16 16:05:54 (206 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-10-16 16:05:55--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[=============

### 1.3 Preparing the data

Training DataFrame

In [3]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=["type", "message"])
train_data.dropna()
train_data

,type,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [4]:
train_data.shape

(4179, 2)

4179 rows and 2 columns

In [5]:
train_data.head()

,type,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [6]:
train_data.tail()

,type,message
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...
4178,ham,tell my bad character which u dnt lik in me. ...


In [7]:
train_data.describe()

,type,message
count,4179,4179
unique,2,3935
top,ham,"sorry, i'll call later"
freq,3619,22


In [8]:
train_data.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
4174    False
4175    False
4176    False
4177    False
4178    False
Length: 4179, dtype: bool

Test DataFrame

In [9]:
test_data = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "message"])
test_data.dropna()
test_data

,type,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [10]:
test_data.shape

(1392, 2)

1392 rows and 2 columns

In [11]:
test_data.head()

,type,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [12]:
test_data.tail()

,type,message
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...
1391,spam,not heard from u4 a while. call 4 rude chat pr...


In [13]:
test_data.describe()

,type,message
count,1392,1392
unique,2,1349
top,ham,"sorry, i'll call later"
freq,1205,8


In [14]:
test_data.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1387    False
1388    False
1389    False
1390    False
1391     True
Length: 1392, dtype: bool

In [15]:
# Get the number of examples in each of the two datasets
num_train_examples = len(train_data)
num_test_examples = len(test_data)

print("Training Entries: {}, Test Entries: {}".format(num_train_examples, num_test_examples))

Training Entries: 4179, Test Entries: 1392


In [68]:
# Prepare labels for training using Pandas' factorize method
train_data["type"] = pd.factorize(train_data["type"])[0]
train_data.head(10)

,type,message
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...
5,0,in xam hall boy asked girl tell me the startin...
6,0,genius what's up. how your brother. pls send h...
7,0,they finally came to fix the ceiling.
8,1,urgent! call 09066350750 from your landline. y...
9,0,now that you have started dont stop. just pray...


In [69]:
# Prepare labels for testing using Pandas' factorize method
test_data["type"] = pd.factorize(test_data["type"])[0]
test_data.head()

,type,message
0,0,i am in hospital da. . i will return home in e...
1,0,"not much, just some textin'. how bout you?"
2,0,i probably won't eat at all today. i think i'm...
3,0,don‘t give a flying monkeys wot they think and...
4,0,who are you seeing?


In [70]:
train_labels = train_data["type"].values
train_slices = tf.data.Dataset.from_tensor_slices((train_data["message"].values, train_labels))

In [71]:
test_labels = test_data["type"].values
test_slices = tf.data.Dataset.from_tensor_slices((test_data["message"].values, test_labels))

In [72]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_slices = train_slices.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_slices = test_slices.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [73]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

In [74]:
# Create a TextVectorization layer for our model
text_vectorizer = TextVectorization(max_tokens=1000, output_mode="int", output_sequence_length=1000)
text_vectorizer.adapt(train_slices.map(lambda text, label: text))


In [75]:
# SHow the first 10 words in the vocabulary
text_vectorizer.get_vocabulary()[:10]

['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in']

In [97]:
# Create a model that achieves statistical power to beat the baseline, but is not overfit, regularized, and has a reasonable training time
embedding_dim = 16

model = tf.keras.Sequential([
    text_vectorizer,
    layers.Embedding(len(text_vectorizer.get_vocabulary()), 64, mask_zero=True),
    layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    layers.Bidirectional(tf.keras.layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1)])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_4 (TextV  (None, 1000)             0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 1000, 64)          64000     
                                                                 
 bidirectional_1 (Bidirectio  (None, 1000, 128)        66048     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                      

In [98]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])



In [ ]:
# Train the model
history = model.fit(train_slices, epochs=10,
                    validation_data=test_slices,
                    validation_steps=30)

In [85]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_slices)

44/44 [==============================] - 0s 3ms/step - loss: nan - binary_accuracy: 0.8657


In [96]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  predictions = model.predict([pred_text])
  print(predictions)
  p = predictions[0][0]
  return [p, "ham" if p < 0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 30ms/step
[[nan]]
[nan, 'spam']


In [95]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 30ms/step
[[nan]]
1/1 [==============================] - 0s 29ms/step
[[nan]]
1/1 [==============================] - 0s 31ms/step
[[nan]]
1/1 [==============================] - 0s 32ms/step
[[nan]]
1/1 [==============================] - 0s 32ms/step
[[nan]]
1/1 [==============================] - 0s 32ms/step
[[nan]]
1/1 [==============================] - 0s 30ms/step
[[nan]]
You haven't passed yet. Keep trying.
